In [ ]:
# Cài đặt thư viện cần thiết
!pip install pydub
!apt install ffmpeg

from pydub import AudioSegment
from google.colab import files
import os
import zipfile
import csv


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
# Thư mục làm việc
input_folder = '/content/uploaded_mp3'
output_folder = '/content/processed_wav'
os.makedirs(input_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# Tải file MP3 từ máy
print("📤 Vui lòng chọn các file .mp3 từ máy tính để tải lên:")
uploaded = files.upload()


📤 Vui lòng chọn các file .mp3 từ máy tính để tải lên:


In [ ]:
# Lưu file vào thư mục input
for filename in uploaded.keys():
    with open(os.path.join(input_folder, filename), 'wb') as f:
        f.write(uploaded[filename])


In [ ]:
# Danh sách thông tin cho 2 file CSV
processed_data = []
skipped_data = []


In [ ]:
# Hàm xử lý từng file
def process_audio(input_path, output_path, original_name):
    try:
        audio = AudioSegment.from_file(input_path, format="mp3")
        original_duration_sec = round(len(audio) / 1000, 2)

        # Nếu file dưới 3 giây, bỏ qua và ghi vào danh sách skipped
        if original_duration_sec < 3.0:
            skipped_data.append([original_name, original_duration_sec])
            print(f"⚠️ Bỏ qua {original_name} (dưới 3s: {original_duration_sec}s)")
            return

        # Chỉ xử lý nếu dài hơn 4s
        if original_duration_sec >= 4.0:
            audio = audio.set_channels(1)
            audio = audio.set_frame_rate(16000)

            # Cắt đúng 4 giây
            audio = audio[:4000]

            # Xuất file WAV
            audio.export(output_path, format="wav")

            processed_duration_sec = round(len(audio) / 1000, 2)
            sample_rate = audio.frame_rate
            channels = audio.channels
            file_size_kb = round(os.path.getsize(output_path) / 1024, 2)

            processed_data.append([
                original_name,
                os.path.basename(output_path),
                original_duration_sec,
                processed_duration_sec,
                sample_rate,
                channels,
                file_size_kb
            ])
            print(f"✅ Đã xử lý: {original_name} -> {os.path.basename(output_path)}")
        else:
            print(f"⏭ Bỏ qua {original_name} (dưới 4s)")
    except Exception as e:
        print(f"❌ Lỗi khi xử lý {original_name}: {e}")

In [ ]:
# Xử lý tất cả file
for filename in os.listdir(input_folder):
    if filename.endswith(".mp3"):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename.replace(".mp3", ".wav"))
        process_audio(input_path, output_path, filename)


⏭ Bỏ qua common_voice_vi_21833315.mp3 (dưới 4s)
✅ Đã xử lý: common_voice_vi_23847384 (1).mp3 -> common_voice_vi_23847384 (1).wav
✅ Đã xử lý: common_voice_vi_23078524 (1).mp3 -> common_voice_vi_23078524 (1).wav
⚠️ Bỏ qua common_voice_vi_23078577 (1).mp3 (dưới 3s: 2.98s)
⏭ Bỏ qua common_voice_vi_23322815.mp3 (dưới 4s)
✅ Đã xử lý: common_voice_vi_22750202 (1).mp3 -> common_voice_vi_22750202 (1).wav
⚠️ Bỏ qua common_voice_vi_22427362 (1).mp3 (dưới 3s: 2.54s)
✅ Đã xử lý: common_voice_vi_23669908 (1).mp3 -> common_voice_vi_23669908 (1).wav
⚠️ Bỏ qua common_voice_vi_23848845.mp3 (dưới 3s: 2.59s)
✅ Đã xử lý: common_voice_vi_23695846.mp3 -> common_voice_vi_23695846.wav
✅ Đã xử lý: common_voice_vi_22855879.mp3 -> common_voice_vi_22855879.wav
⚠️ Bỏ qua common_voice_vi_23727885 (1).mp3 (dưới 3s: 2.5s)
⏭ Bỏ qua common_voice_vi_23406213.mp3 (dưới 4s)
⏭ Bỏ qua common_voice_vi_22402415.mp3 (dưới 4s)
⏭ Bỏ qua common_voice_vi_23334648 (1).mp3 (dưới 4s)
⏭ Bỏ qua common_voice_vi_23753756 (1).mp3 (dưới 4s)

In [ ]:
# Ghi CSV cho các file đã xử lý (CÓ THÊM STT)
csv1 = '/content/processed_audio_metadata.csv'
with open(csv1, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([
        'STT',
        'Tên file gốc',
        'Tên file WAV sau xử lý',
        'Thời lượng gốc (giây)',
        'Thời lượng sau xử lý (giây)',
        'Sample Rate (Hz)',
        'Số kênh',
        'Kích thước WAV (KB)'
    ])
    for idx, row in enumerate(processed_data, start=1):
        writer.writerow([idx] + row)


In [ ]:
# Ghi danh sách file bị bỏ qua (dưới 3s)
csv2 = '/content/skipped_files_under_3s.csv'
with open(csv2, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Tên file gốc', 'Thời lượng gốc (giây)'])
    writer.writerows(skipped_data)

print("✅ Đã tạo 2 file CSV (1 file chứa metadata, 1 file chứa danh sách file bị bỏ qua).")


✅ Đã tạo 2 file CSV (1 file chứa metadata, 1 file chứa danh sách file bị bỏ qua).


In [ ]:
# Nén toàn bộ file WAV và 2 file CSV
zip_path = '/content/processed_audio_and_metadata.zip'
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for file in os.listdir(output_folder):
        zipf.write(os.path.join(output_folder, file), arcname=file)
    zipf.write(csv1, arcname='processed_audio_metadata.csv')
    zipf.write(csv2, arcname='skipped_files_under_3s.csv')


In [ ]:
# Tải file zip về máy
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>